In [9]:
import re
import geopandas as gpd
from shapely.geometry import Point

# Leer el archivo de texto
file_path = 'lista_lia_gmaps.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Expresión regular para encontrar coordenadas entre paréntesis
coord_pattern = re.compile(r'\((-?\d+\.\d+),\s*(-?\d+\.\d+)\)')

data = []
i = 0
marker_count = 1

while i < len(lines):
    line = lines[i].strip()
    match = coord_pattern.search(line)
    
    if match:
        lat = float(match.group(1))
        lon = float(match.group(2))
        name = None
        
        # Buscar el nombre en las siguientes líneas válidas
        for j in range(i + 1, len(lines)):
            next_line = lines[j].strip()
            if len(next_line) >= 3 and next_line != "Marcador" and not coord_pattern.search(next_line):
                name = next_line
                break

        # Si no se encuentra un nombre válido, asignar un nombre genérico
        if not name:
            name = f'Marcador_{marker_count}'
            marker_count += 1
        
        data.append((name, lat, lon))
    
    i += 1

# Crear un GeoDataFrame con la columna de geometría
gdf = gpd.GeoDataFrame(data, columns=['name', 'latitude', 'longitude'], geometry=gpd.points_from_xy([x[2] for x in data], [x[1] for x in data]))

# Especificar la proyección EPSG:4326
gdf.set_crs(epsg=4326, inplace=True)

# Guardar el GeoDataFrame en un archivo SHP
output_file = 'marcadores.shp'
gdf.to_file(output_file, driver='ESRI Shapefile')

print(f'Archivo SHP creado: {output_file}')


Archivo SHP creado: marcadores.shp
